In [26]:
import jieba
import pandas as pd
import numpy as np
from collections import Counter
from scipy.sparse import coo_matrix
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier

In [27]:
train_path = 'weibo_3_moods_train.csv'
test_path = 'weibo_3_moods_test.csv'
types = {0: '喜悦', 1: '愤怒', 2: '哀伤'}
pd_train = pd.read_csv(train_path)
pd_test = pd.read_csv(test_path)

In [28]:
# 打印训练集行数
print('训练集数目（总体）：%d' % pd_train.shape[0])
# 打印测试集函数
print('测试集数目（总体）：%d' % pd_test.shape[0])

训练集数目（总体）：289395
测试集数目（总体）：72349


In [29]:
#中文分词
train_words = []
test_words = []
train_labels = []
test_labels = []
stopwords = ["[", "]", "）", "（", ")", "(", "【", "】", "！", "，", "$",
             "·", "？", ".", "、", "-", "—", ":", "：", "《", "》", "=",
             "。", "…", "“", "?", "”", "~", " ", "－", "+", "\\", "‘",
             "～", "；", "’", "...", "..", "&", "#",  "....", ",",
             "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"
             "的", "和", "之", "了", "哦", "那", "一个",  ]

In [30]:
for line in range(len(pd_train)):
    dict_label = pd_train['label'][line]
    dict_content = str(pd_train['content'][line]) #float=>str
    #print(dict_label,dict_content)
    cut_words = ""
    data = dict_content.strip("\n")
    data = data.replace(",", "")    #一定要过滤符号 ","否则多列  注意到是英文逗号。csv中 英文逗号表示分列。
    seg_list = jieba.cut(data, cut_all=False)
    for seg in seg_list:
        if seg not in stopwords:
            cut_words += seg + " "
    #print(cut_words)
    label = dict_label
    train_labels.append(label)
    train_words.append(cut_words)
print(len(train_labels), len(train_words)) #289395 289395
print(train_labels[:5])
print(train_words[:5])

289395 289395
[1, 1, 0, 0, 3]
['最 讨厌 有人 跟 我 说 互相 关注 下 吧 我 想 关注 你 自然 会 关注 我 不想 就 别来 烦 我 姐姐 我 不在乎 粉丝 多少 ', '回复 千年老二 恐韩 不服 不行 广州 亚运会 射箭 女团 决赛 由程明 张云录 祝 珊 珊 组成 的 中国队 通过 两轮 加赛 以 环 的 劣势 不 敌 韩国队 屈居亚军 韩国队 夺冠 ', '就是 嘛 小心 被 跨省 追捕 住 在 中南海 不行 驻京办 无孔不入 我 借 新浪 围脖 这个 平台 友情 提醒 一下 胡主席 一定 要 注意 自己 的 身体 一定 不要 乱发 敏感 信息 一定 不要 辜负 党和人民 的 重托 胡锦涛 主席 在 人民网 开通 微博 这是 多么 振奋人心 的 消息 太 激动 这是 里程碑式 标志性 事件 互联网 将 进入 划时代 崭新 的 时代 ', '翘翘 這 個 很 攪 笑 發 給 你 看看 吧 ', '树洞 啊 树洞 我 难过 死 你 不 保佑 我 也 就算 居然 还 让 我 分到 我 不想 去 的 班级 居然 还是 在 三楼 而且 居然 在 厕所 旁边 我 难过 啊 求 安慰 ']


In [31]:
for line in range(len(pd_test)):
    dict_label = pd_test['label'][line]
    dict_content = str(pd_test['content'][line])
    cut_words = ""
    data = dict_content.strip("\n")
    data = data.replace(",", "")
    seg_list = jieba.cut(data, cut_all=False)
    for seg in seg_list:
        if seg not in stopwords:
            cut_words += seg + " "

    #  text = " ".join(list(jieba.cut(text)))
    label = dict_label
    test_labels.append(label)
    test_words.append(cut_words)
print(len(test_labels),len(test_words))
print(test_labels[:5])
print(test_words[:5])

72349 72349
[3, 0, 0, 0, 0]
['星期五 去將 軍 奥翠林 的 分店 拉客 加油 呀 因为 老細 会落 來 睇 場 睡 啦 要 早起 搭車 又系 疲勞 的 一天 開 始 ', '回复 哈哈 谢谢 开心 就 好 大 妹子 好 诗 远山 纵暮霞 近林 横霭 桠 急舟 鳞 波长 慢火 渔 窗花 一起 玩 开心 就 好 呵呵 好 好 拜服 飞霞 相映 远山 分 长堤 稀树 晓烟沉 清风徐来 野村 近 一舟 摇去 半 湖痕 ', '回复 我 是 说 这么 多 花花 是 干 啥 的 不干 啥 拍照 啊 呵呵哈 ', '李 卤味 李 卤味 哈哈哈 作业 作业 星期三 就是 星期 山 ! ! ! 幸好 有 贵人相助 下次 去 吃 李 卤味 好不好 哈哈哈哈 哈哈哈 哈哈哈 ', '我 昨天 做空 的 澳元 仍 在手 今晚 有機 會 在 0.9700 水平 之下 收市 到 時 先行 ']


In [32]:
#TFIDF计算
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
vectorizer = CountVectorizer(min_df=100)   #MemoryError控制参数
# vectorizer = TfidfVectorizer(min_df=100)
#该类会统计每个词语的tf-idf权值
transformer = TfidfTransformer()
# tfidf = vectorizer.fit_transform(train_words+test_words)
#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵

# 合并在一起进行了tf-idf
tfidf = transformer.fit_transform(vectorizer.fit_transform(train_words+test_words))
print(tfidf[:5])
print(tfidf.shape)

  (0, 7089)	0.2653697319158461
  (0, 6766)	0.271911936824012
  (0, 6394)	0.20697302118474042
  (0, 4899)	0.22226864932059226
  (0, 3100)	0.2545146290153176
  (0, 2735)	0.23398051065790837
  (0, 1575)	0.611410034200445
  (0, 1042)	0.31248674521223585
  (0, 726)	0.23602128819008097
  (0, 697)	0.3416149878776036
  (1, 7632)	0.2746396605549873
  (1, 6470)	0.36056107542458826
  (1, 3608)	0.25465546388338434
  (1, 2959)	0.3615794555086064
  (1, 2535)	0.1596560807002662
  (1, 1660)	0.3355559683193786
  (1, 1059)	0.33200730583563587
  (1, 880)	0.36297521893155027
  (1, 767)	0.26485442236603435
  (1, 741)	0.3856994452867779
  (2, 7560)	0.17293200707162557
  (2, 7531)	0.26275998506317266
  (2, 7498)	0.09445359768986004
  (2, 7380)	0.16149540897173548
  (2, 6757)	0.09845337874080341
  :	:
  (2, 3363)	0.16818638259815935
  (2, 2725)	0.1729879609810629
  (2, 2566)	0.13907365267846508
  (2, 2073)	0.20220188173164827
  (2, 1388)	0.16662941040878823
  (2, 1121)	0.2298207531081246
  (2, 1044)	0.1922639

In [33]:

#获取词袋模型中的所有词语
word = vectorizer.get_feature_names()
print(word[:15])
print("单词数量:", len(word))

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '10000', '10086']
单词数量: 8194


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
#将tf-idf矩阵抽取 元素w[i][j]表示j词在i类文本中的tf-idf权重
X = coo_matrix(tfidf, dtype=np.float32).toarray()  #稀疏矩阵
print(X.shape)
print(X[:10])

(361744, 8194)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [35]:
X_train = X[:len(train_labels)]
X_test = X[len(train_labels):]
y_train = train_labels
y_test = test_labels

In [36]:
#分类模型
clf = MultinomialNB()
#clf = svm.LinearSVC()
#clf = LogisticRegression(solver='liblinear')
#clf = RandomForestClassifier(n_estimators=10)
#clf = neighbors.KNeighborsClassifier(n_neighbors=7)
#clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

MultinomialNB()

In [37]:
print('模型的准确度:{}'.format(clf.score(X_test, y_test)))

模型的准确度:0.575875271254613


In [38]:
pre = clf.predict(X_test)

In [39]:
print(classification_report(y_test, pre, digits=4))

              precision    recall  f1-score   support

           0     0.5979    0.9793    0.7425     39912
           1     0.5274    0.1340    0.2137     10402
           2     0.2783    0.0592    0.0976     10933
           3     0.2678    0.0484    0.0819     11102

    accuracy                         0.5759     72349
   macro avg     0.4179    0.3052    0.2839     72349
weighted avg     0.4888    0.5759    0.4676     72349



In [40]:
# 0: 喜悦, 1: 愤怒, 2: 厌恶, 3: 低落